# Dipendenze

In [1]:
%pip install --quiet --upgrade langchain-sqlserver python-dotenv pypdf langchain-openai langchain langchain-core langchain-text-splitters langchain-community langgraph

Note: you may need to restart the kernel to use updated packages.


### LangSmith Setup

In [2]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"

In [3]:
#Env File

from dotenv import load_dotenv

load_dotenv("intro.env");

### OAI LLM Setup

In [4]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    openai_api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
)

### Embeddings setup

In [5]:
from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
    azure_endpoint=os.getenv("AZURE_OPENAI_EMBEDDINGS_ENDPOINT"),
    azure_deployment=os.getenv("AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT_NAME"),
    openai_api_version=os.getenv("AZURE_OPENAI_EMBEDDINGS_API_VERSION"),
)

### VectorStorage

In [6]:
from langchain_core.vectorstores import InMemoryVectorStore

local_vector_store = InMemoryVectorStore(embeddings)

### DB Connection

In [7]:
#connection string

_CONNECTION_STRING = (
    "Driver={{ODBC Driver 18 for SQL Server}};"
    "Server=tcp:inft-dev.database.windows.net,1433;"
    "Database=inft;"
    "Uid={username};"
    "Pwd={password};"
    "Encrypt=yes;"
    "TrustServerCertificate=no;"
    "Connection Timeout=30;"
).format(
    username=os.getenv("SQL_SERVER_USER"),
    password=os.getenv("SQL_SERVER_PASSWORD")
)

In [8]:
from langchain_sqlserver import SQLServer_VectorStore
from langchain_community.vectorstores.utils import DistanceStrategy

sql_vector_store = SQLServer_VectorStore(
    connection_string=_CONNECTION_STRING,
    distance_strategy=DistanceStrategy.COSINE,  # optional, if not provided, defaults to COSINE
    embedding_function=embeddings,  # you can use different embeddings provided in LangChain
    embedding_length=1536,
    table_name="dbo.Web3",  # using table with a custom name
)

# Progetto

### 1 - Retrieval

In [9]:
DIR_PATH = "./data"

In [10]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
import pypdf

loader = PyPDFDirectoryLoader(DIR_PATH)
docs = loader.load()

In [11]:
print(len(docs[0].page_content))

1375


In [12]:
#Chunking

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # chunk size (characters)
    chunk_overlap=100,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)

all_splits = text_splitter.split_documents(docs)

print(f"Split into {len(all_splits)} sub-documents.")

Split into 36 sub-documents.


In [13]:
ids = local_vector_store.add_documents(documents=all_splits)

for i in range(3):
    print(ids[i])

e2ca670d-8032-41c1-ab26-057a65689100
a60f62f5-350a-4ab9-9fee-28fbff3bbfeb
2768ce01-ddc1-4700-8b2d-c0cf5e8544e3


### 2 - Retrieval & Generation

In [14]:
#Initial State

answer_history = []
user_history = []

In [15]:
from langchain import hub
from langchain_core.prompts import PromptTemplate

prompt = hub.pull("rlm/rag-prompt")

In [16]:
template = """
Use the following pieces of context, User History and Answer History to answer the Question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

User History: {user_history}

Answer History: {answer_history}

Helpful Answer:
"""

In [17]:
prompt = PromptTemplate.from_template(template)

In [18]:
from langchain_core.documents import Document
from typing_extensions import List, TypedDict

class State(TypedDict):

    question: str
    context: List[Document]
    answer: str
    answer_history: List[str]
    user_history: List[str]

In [22]:
def retrieve(state: State):
    # Recupera risultati da entrambi i vector store con score
    local_results = local_vector_store.similarity_search_with_score(state["question"])
    sql_results = sql_vector_store.similarity_search_with_score(state["question"])
    
    # Unisci i risultati e deduplica in base al contenuto della pagina
    all_results = local_results + sql_results

    #deduplication
    seen = set()
    deduped_results = []
    for doc, score in all_results:
        # Usa il contenuto come chiave per la deduplica
        key = doc.page_content.strip()
        if key not in seen:
            seen.add(key)
            deduped_results.append((doc, score))

    #sorting
    deduped_results.sort(key=lambda x: x[1])
    
    # Restituisci tutti i risultati deduplicati con score
    return {"context": [doc for doc, score in deduped_results], "results_with_score": deduped_results}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])

    prompt_text = prompt.format(
        question=state["question"], 
        context=docs_content,
        user_history=state["user_history"],
        answer_history=state["answer_history"]
    )
    
    response = llm.invoke(prompt_text)

    #History save
    answer_history = state.get("answer_history", [])
    answer_history.append(response.content)

    user_history = state.get("user_history", [])
    user_history.append(state["question"])

    return {"answer": response.content, 
            "answer_history": answer_history, 
            "user_history": user_history}

In [20]:
from langgraph.graph import START, StateGraph

graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")

graph = graph_builder.compile()

In [23]:
while True:
    user_input = input("Please insert your message! (digita 'exit' per uscire) ")
    if user_input.lower() == "exit":
        break

    # Invoca il grafo passando le history aggiornate
    result = graph.invoke({
        "question": user_input,
        "answer_history": answer_history,
        "user_history": user_history
    })

    print(f'Answer: {result["answer"]}\n')

    # Aggiorna le history per la prossima iterazione
    answer_history = result.get("answer_history", [])
    user_history = result.get("user_history", [])


Answer: Non è possibile determinare il numero di elementi presenti nella tabella SQL basandosi solo sui dati forniti nel contesto. Non ci sono riferimenti espliciti a una tabella SQL o al numero delle sue righe o colonne. Grazie per asking!

Answer: Non ci sono riferimenti nel contesto su specifiche tabelle presenti su SQL Server che puoi interrogare. L'informazione sulle tabelle disponibili non è indicata nei dati forniti. Grazie per asking!

